In [1]:
import numpy as np
import os
import cv2
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, Flatten, MaxPooling2D
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
dataset_path = './data/'

In [3]:
face_data = []
labels = []
class_id = 0

In [4]:
for ix in os.listdir(dataset_path):
    if ix.endswith('.npy'):
        print ix
        data_item = np.load(dataset_path + ix)
        face_data.append(data_item)

        target = class_id * np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)
face_dataset = np.concatenate(face_data, axis=0)
face_labels = np.concatenate(labels, axis=0).reshape((-1, 1))
print face_labels.shape
print face_dataset.shape
print face_labels
print face_dataset

bhumika.npy
shivang.npy
suransh.npy
(3028, 1)
(3028, 10000)
[[0.]
 [0.]
 [0.]
 ...
 [2.]
 [2.]
 [2.]]
[[ 43  45  43 ...  25  31  32]
 [ 49  48  52 ...  51  53  58]
 [ 53  53  53 ...  36  74  81]
 ...
 [141 144 146 ...  49  51  50]
 [140 142 147 ...  50  51  50]
 [126 135 140 ...  50  50  52]]


In [5]:
trainset = np.concatenate((face_dataset, face_labels), axis=1)
print trainset.shape
print trainset

(3028, 10001)
[[ 43.  45.  43. ...  31.  32.   0.]
 [ 49.  48.  52. ...  53.  58.   0.]
 [ 53.  53.  53. ...  74.  81.   0.]
 ...
 [141. 144. 146. ...  51.  50.   2.]
 [140. 142. 147. ...  51.  50.   2.]
 [126. 135. 140. ...  50.  52.   2.]]


In [6]:
face_labels = np_utils.to_categorical(face_labels)
print face_labels

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [7]:
face_dataset = face_dataset/255.0

In [8]:
print face_dataset

[[0.16862745 0.17647059 0.16862745 ... 0.09803922 0.12156863 0.1254902 ]
 [0.19215686 0.18823529 0.20392157 ... 0.2        0.20784314 0.22745098]
 [0.20784314 0.20784314 0.20784314 ... 0.14117647 0.29019608 0.31764706]
 ...
 [0.55294118 0.56470588 0.57254902 ... 0.19215686 0.2        0.19607843]
 [0.54901961 0.55686275 0.57647059 ... 0.19607843 0.2        0.19607843]
 [0.49411765 0.52941176 0.54901961 ... 0.19607843 0.19607843 0.20392157]]


In [9]:
face_dataset = face_dataset.reshape((-1,100,100,1))
print face_dataset

[[[[0.16862745]
   [0.17647059]
   [0.16862745]
   ...
   [0.18039216]
   [0.17254902]
   [0.19215686]]

  [[0.16470588]
   [0.16862745]
   [0.16470588]
   ...
   [0.18039216]
   [0.18431373]
   [0.2       ]]

  [[0.16078431]
   [0.15686275]
   [0.16470588]
   ...
   [0.18823529]
   [0.17647059]
   [0.18431373]]

  ...

  [[0.09019608]
   [0.09411765]
   [0.09411765]
   ...
   [0.1372549 ]
   [0.11764706]
   [0.11764706]]

  [[0.07843137]
   [0.07843137]
   [0.08627451]
   ...
   [0.1254902 ]
   [0.12941176]
   [0.13333333]]

  [[0.06666667]
   [0.08235294]
   [0.07058824]
   ...
   [0.09803922]
   [0.12156863]
   [0.1254902 ]]]


 [[[0.19215686]
   [0.18823529]
   [0.20392157]
   ...
   [0.21568627]
   [0.21176471]
   [0.23921569]]

  [[0.20392157]
   [0.19215686]
   [0.19215686]
   ...
   [0.22745098]
   [0.20392157]
   [0.24313725]]

  [[0.19215686]
   [0.18823529]
   [0.18823529]
   ...
   [0.24313725]
   [0.23921569]
   [0.25098039]]

  ...

  [[0.07843137]
   [0.0745098 ]
   [0.0

In [10]:
model = Sequential()

## Shape Here - (b, 100, 100, 1)
model.add(Convolution2D(32, kernel_size=(10, 10), input_shape=(100, 100, 1)))
model.add(Activation('relu'))
## Shape Here - (b, 91, 91, 32)
model.add(Convolution2D(16, kernel_size=(10, 10)))
model.add(Activation('relu'))
## Shape Here - (b, 82, 82, 16)
model.add(MaxPooling2D(pool_size=(2, 2)))
## Shape Here - (b, 41, 41, 16)
model.add(Convolution2D(8, (22, 22)))
model.add(Activation('relu'))
## Shape Here - (b, 20, 20, 8)
model.add(Convolution2D(8, (13, 13)))
model.add(Activation('relu'))
## Shape Here - (b, 8, 8, 8)
model.add(Flatten())
## Shape Here - (b, 512)
model.add(Dense(256))
model.add(Activation('relu'))
## Shape Here - (b, 256)
model.add(Dense(3))
## Shape Here - (b, 10)
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 91, 91, 32)        3232      
_________________________________________________________________
activation_1 (Activation)    (None, 91, 91, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 82, 82, 16)        51216     
_________________________________________________________________
activation_2 (Activation)    (None, 82, 82, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 41, 41, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 8)         61960     
_________________________________________________________________
activation_3 (Activation)    (None, 20, 20, 8)         0         
__________

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(face_dataset, face_labels, epochs=5, batch_size=50)

Epoch 1/5
3028/3028 [==============================] - 474s 157ms/step - loss: 0.6048 - acc: 0.7097
Epoch 2/5
3028/3028 [==============================] - 417s 138ms/step - loss: 0.0166 - acc: 0.9964
Epoch 3/5
3028/3028 [==============================] - 417s 138ms/step - loss: 0.0128 - acc: 0.9974
Epoch 4/5
3028/3028 [==============================] - 3769s 1s/step - loss: 0.0050 - acc: 0.9977
Epoch 5/5
3028/3028 [==============================] - 443s 146ms/step - loss: 0.0042 - acc: 0.9993


In [14]:
model.save('Face_Recogizer.h5')